In [46]:
import torch
import torch.nn as nn
import sys
from config import input_size, proposalN, channels
from utils.read_dataset import read_dataset
from utils.auto_laod_resume import auto_load_resume
from networks.model import MainNet
from torchvision import transforms
import json
import imageio
import numpy as np
from PIL import Image

import os

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [47]:
DEVICE = torch.device("cpu")


In [48]:
root = './datasets/FGVC-aircraft'  # dataset path
# model path
pth_path = "./models/air_epoch146.pth"
num_classes = 100

batch_size = 10

In [49]:
_, testloader = read_dataset(input_size, batch_size, root, "Aircraft")

Loading Aircraft trainset
Loading Aircraft testset


In [50]:
model = MainNet(proposalN=proposalN, num_classes=num_classes, channels=channels)

In [51]:
model = model.to(DEVICE)
criterion = nn.CrossEntropyLoss()

In [52]:
if os.path.exists(pth_path):
    epoch = auto_load_resume(model, pth_path, status='test')
else:
    sys.exit('There is not a pth exist.')

Load model from ./models/air_epoch146.pth
Resume from ./models/air_epoch146.pth


In [53]:
with open('id_label.json', 'r') as f:
    id_label_data = json.load(f)

id_label = {int(k): v for k, v in id_label_data.items()}

In [54]:
def preprocess_image(img):
    if len(img.shape) == 2:
        img = np.stack([img] * 3, 2)
    img = Image.fromarray(img, mode='RGB')
    img = transforms.Resize((448, 448), Image.BILINEAR)(img)
    img = transforms.ToTensor()(img)
    img = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])(img)
    img = img.unsqueeze(0)
    return img

img = preprocess_image(imageio.imread("image.png"))

model.eval()

with torch.no_grad():
    output = model(img, None, None, "test", "cpu")[-2:]
    pred = output[0].max(1, keepdim=True)[1]
    print(pred.numpy()[0][0], id_label[pred.numpy()[0][0]])

33 A380
